We have a lot of independent scripts here that in some time of the development was usefull.

In sequence:

- create overlaped video
- Organize dataset: organize from sideseeing format to DHF1K format

install:
`pip install opencv-python tqdm`

In [1]:
frames_to_save = ['00100.jpg', '00500.jpg', '01000.jpg', '01200.jpg']
# frames_to_save = ['00001.jpg', '00050.jpg', '00100.jpg', '00200.jpg']

In [2]:
"""
This code creates a video by overlaying masks onto corresponding images.
It reads image-mask pairs from specified folders, applies a semi-transparent heatmap, and combines them into a video.
The output video is saved with a given frame rate and resolution and at a given path.
"""
import cv2
import os
import numpy as np
from tqdm import tqdm

# Paths to images and masks
image_folder = "/scratch/suayder/urbanaccess_dhf1/val/0004/images/" 
# mask_folder = "/scratch/suayder/urbanaccess_dhf1_vcentered/val/0004/maps/" # ground truth
# mask_folder = "/home/suayder/JBCS-paper/STSANet/results/tactile_side/"
# mask_folder = "/home/suayder/JBCS-paper/TMFI-Net/results_tmfiside/0004/"
mask_folder = "/home/suayder/JBCS-paper/ViNet/ViNet/output/centered/0004/"
output_video = "output.mp4" 

# Video parameters
frame_rate = 30  # Frames per second
overlay_color = (0, 255, 0)  # Green overlay for mask (can be changed)
alpha = 0.3  # Transparency of the mask overlay

# Get sorted list of images and masks (assumes same filenames)
image_files = sorted([f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))])
mask_files = sorted([f for f in os.listdir(mask_folder) if f.endswith(('.png', '.jpg', '.jpeg'))])

image_files = [im for im in image_files if im in mask_files] # REMOVER

# Ensure image and mask count match
if len(image_files) != len(mask_files):
    raise ValueError("Number of images and masks do not match!")

# Read first image to get frame size
sample_image = cv2.imread(os.path.join(image_folder, image_files[0]))
height, width, _ = sample_image.shape

# Initialize video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(output_video, fourcc, frame_rate, (width, height))

# Process each image and corresponding mask
pbar = tqdm(zip(image_files, mask_files), total=len(image_files), desc="Processing images")
for img_name, mask_name in pbar:
    pbar.set_description("Processing %s" % img_name)
    img_path = os.path.join(image_folder, img_name)
    mask_path = os.path.join(mask_folder, mask_name)

    # Read image and mask
    image = cv2.imread(img_path)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  # Read mask as grayscale
    mask = cv2.resize(mask, (image.shape[1], image.shape[0]))

    if image is None or mask is None:
        print(f"Skipping {img_name} due to loading error.")
        continue

    # Normalize mask to 0-255 range if needed
    mask = cv2.normalize(mask, None, 0, 255, cv2.NORM_MINMAX)

    # Apply a heatmap color map
    heatmap = cv2.applyColorMap(mask, cv2.COLORMAP_JET)
    overlayed_image = cv2.addWeighted(image, 1, heatmap, alpha, 0)
    if img_name in frames_to_save:
        cv2.imwrite(img_name.replace('.png', '.jpg'), overlayed_image)
    # Write frame to video
    video_writer.write(overlayed_image)


# Release video writer
video_writer.release()
print(f"Video saved as {output_video}")


Processing 02090.jpg: 100%|█████████████████| 2090/2090 [00:55<00:00, 37.54it/s]


Video saved as output.mp4


In [4]:
"""
Organize dataset: organize from sideseeing format to DHF1K format
this script will make a copy of images/ fixations/ and maps/ to a folder, you have the choice to say if is train or test data.

In summary, will transform from this:

src_dir/
  ├── data/
  │   ├── Area1/
  │   │   ├── Block1/
  │   │   │   ├── fixations/
  │   │   │   ├── images/
  │   │   │   ├── maps/

to this:

processed_dataset/
  ├── train/
  │   ├── Area1#Block1/
  │   │   ├── fixations/
  │   │   ├── images/
  │   │   ├── maps/

you can apply rename_name2numbers to have a dataset like this

processed_dataset/
  ├── train/
  │   ├── 0000/
  │   │   ├── fixations/
  │   │   ├── images/
  │   │   ├── maps/
  │   ├── 0001/

"""

import os
import concurrent.futures
import shutil
from pathlib import Path

def reorganize_dataset(src_dir, dest_dir, split='train'):
    """
    Reorganizes the dataset into the desired format.

    - split (str): Either 'train' or 'val' to specify the split.
    """

    src_path = Path(src_dir)
    dest_path = Path(dest_dir) / split
    
    if not src_path.exists():
        print(f"Source directory '{src_dir}' does not exist.")
        return

    # Create destination directory if it doesn't exist
    dest_path.mkdir(parents=True, exist_ok=True)

    # Iterate through areas and blocks in the dataset
    for area_dir in src_path.glob("data/*"):
        if area_dir.is_dir():
            for block_dir in area_dir.glob("*"):
                if block_dir.is_dir():
                    # Construct the new folder name
                    new_folder_name = f"{area_dir.name}#{block_dir.name}"
                    new_folder_path = dest_path / new_folder_name
                    new_folder_path.mkdir(parents=True, exist_ok=True)

                    # Move required subdirectories/files
                    for subfolder in ['fixations', 'images', 'maps']:
                        subfolder = subfolder+'_centered' if subfolder!='images' else subfolder
                        src_subfolder = block_dir / subfolder
                        if src_subfolder.exists():
                            dest_subfolder = new_folder_path / subfolder
                            dest_subfolder.mkdir(parents=True, exist_ok=True)

                            def copy_file(file):
                                if file.is_file():
                                    shutil.copy(file, dest_subfolder)

                            with concurrent.futures.ThreadPoolExecutor() as executor:
                                executor.map(copy_file, src_subfolder.iterdir())
                            # for file in src_subfolder.iterdir():
                            #     if file.is_file():
                            #         shutil.copy(file, dest_subfolder)

    print(f"Dataset reorganized under '{dest_path}'.")

def rename_name2numbers(path, split="train"):

    path = Path(path) / split
    names = os.listdir(path)

    for i, name in enumerate(names):
        (path / name).rename(path / str(i).zfill(4))

    print('rename_name2numbers done')

# usage
source_directory = "/scratch/suayder/urbanaccess"
destination_directory = "/scratch/suayder/urbanaccess_dhf1_vcentered"
reorganize_dataset(source_directory, destination_directory, split='train')
rename_name2numbers(destination_directory)

Dataset reorganized under '/scratch/suayder/urbanaccess_dhf1_vcentered/train'.
rename_name2numbers done
